In [7]:
import pandas as pd
import numpy as np
import os
from PIL import Image, ImageDraw

In [8]:
data = pd.read_csv('10fold/pneumonia-test-0', sep=';')
data2 = pd.read_csv('10fold/pneumonia-train-0', sep=';')
data.append(data2, ignore_index=True)
data.reset_index(inplace=True, drop=True)

,x,y,width,height,Target
count,900.000000,900.000000,900.000000,900.000000,3001.00000
mean,401.865556,379.456667,213.986667,316.087778,0.29990
std,205.179806,149.569636,57.777024,149.867380,0.45829
min,6.000000,26.000000,59.000000,46.000000,0.00000
25%,216.750000,258.000000,173.750000,201.000000,0.00000
50%,343.500000,386.000000,214.000000,285.000000,0.00000
75%,598.000000,491.000000,253.000000,409.000000,1.00000
max,808.000000,836.000000,413.000000,808.000000,1.00000


In [13]:
def draw_rectangle(image, x_min, y_min, width, height, inner_color=None, out_color='yellow',
                   size=1):
    draw = ImageDraw.Draw(image)
    coordinates = [x_min, y_min, x_min + width + 1, y_min + height + 1]
    draw.rectangle(xy=coordinates, width=size, outline=out_color, fill=inner_color)
    return image

def draw_mask(image, x_min, y_min, width, height):
    draw = ImageDraw.Draw(image)
    coordinates = [x_min, y_min, x_min + width + 1, y_min + height + 1]
    draw.rectangle(xy=coordinates, width=0, outline=None, fill='white')
    return image

def show_grid(images, images_per_line=3):
    #creates a new empty image, RGB mode, and size 400 by 400.
    width, height = images[0].size
    num_lines = int(np.ceil(len(images) / images_per_line))
    grid_width = int(width * images_per_line)
    grid_height = int(height * num_lines)
    background = Image.new('RGB', (grid_width, grid_height))
    
    #Iterate through a 4 by 4 grid with 100 spacing, to place my image
    for i in range(num_lines):
        for j in range(images_per_line):
            idx = i * images_per_line + j
            image = images[idx]
            x = j * width
            y = i * height
            #paste the image at location i,j:
            background.paste(image, (x,y))
    
    return background

In [14]:
image_dir = 'images/train'
output_dir = 'images/masks'
for filename in os.listdir(image_dir):
    # gets bboxes from a image
    name = os.path.basename(filename)
    print(name)
    boxes = data.loc[data['patientId'] == name[:-4]]
    
    image = Image.open(os.path.join(image_dir, name))
    img_width, img_height = image.size
    image = Image.new(mode='RGB', size=(img_width, img_height), color='black')  
    for j in range(len(boxes)):
        example = boxes.iloc[j]
        if example['Target'] == 1:
            image = draw_mask(image=image,
                              x_min=example['x'],
                              y_min=example['y'],
                              width=example['width'],
                              height=example['height'])
    image.save(os.path.join(output_dir, name[:-4] + '.png'))


6495b54d-fceb-4463-8eb0-94dab230e043.jpg
65139428-111f-4e09-82a6-14fafe18cbbe.jpg
patientId    65139428-111f-4e09-82a6-14fafe18cbbe
x                                             NaN
y                                             NaN
width                                         NaN
height                                        NaN
Target                                          0
Name: 893, dtype: object
0174c4bb-28f5-41e3-a13f-a396badc18bd.jpg
patientId    0174c4bb-28f5-41e3-a13f-a396badc18bd
x                                             155
y                                             182
width                                         273
height                                        501
Target                                          1
Name: 2, dtype: object
patientId    0174c4bb-28f5-41e3-a13f-a396badc18bd
x                                             599
y                                             220
width                                         227
height                       